## Setup

See "Scheduling Simulation" for all function descriptions.


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import time
import functions as fn

Configure global simulation parameters.


In [ ]:
## Global variables

cts = np.array([0, 1]) # client types -> normal or emergency
pct = np.array([1, 0]) # distribution of client types -> normal or emergency
# sts = np.array([14, 14, 14, 15, 16]) # service times - for simple numerical testing
# sts = np.random.poisson(lam=14, size=1000) - using poisson distribution

ns = 0.1 # percentage of no-shows
nsims = 100000 # number of simulations
logs = False # do not print simulation logs
lnm = 14 # lognormal mean
lns = 5 # lognormal standard deviation
nm = fn.logn_mean_lns(lnm, lns)[0] # normal mean
ns = fn.logn_mean_lns(lnm, lns)[1] # normal standard deviation
d = 15 # interval time
sts = fn.generate_logn_sts(1, 10000, lnm, lns)[0] # sampling population for service times
sim = np.zeros(12) 
i = np.arange(12, step=2)
sim[i] = 1
sim[-1] = 1
sim[0] = 2 
sim = sim.astype(int) # initial schedule
psm = fn.patient_shift_matrix(sim) 
psm = psm.astype(int) # patient shifting matrix
transf_to_iats = lambda x: fn.transform_schedule_iats(x, d) 
iats_sm = np.apply_along_axis(transf_to_iats, 1, psm) # iats shifting matrix
iats_d_sm = np.diff(iats_sm) # first derivative iats shifting matrix
N = iats_sm.shape[1] # number of patients
T = psm.shape[1] # number of time slots
ctsm = np.random.choice(cts, size = (nsims * N), p=pct).reshape((nsims, N)) # client types matrix
stsm = np.random.lognormal(mean = nm, sigma=ns, size=(nsims * N)).reshape(nsims, N) # service times matrix
nsm = np.random.binomial(n=1, p=1-ns, size = (nsims * N)).reshape(nsims, N) # no-shows matrix
stsm = stsm * nsm # service times matrix with no-shows
print(f'Running {nsims} simulations with {N} patients and {T} timeslots.')

## Run simulations


In [ ]:
# Simulations
results = []
for iats in iats_sm:
  tic = time.time()
  simreswt = [None] * nsims # array for saving waiting times
  simresln = 0 # array for saving lateness
  
  for i in range(nsims):
    experiment = fn.simulate_crn(iats, ctsm[i], stsm[i], logs=logs)
    simreswt[i] = experiment[0].mean()
    ln = max(0, (experiment[1] - d))
    simresln += ln
    
  mwt = np.array(simreswt).mean()
  mln = simresln / nsims
  loss = 0.5 * mwt + 0.5 * mln
  toc = time.time()
  result = {
      "schedule": fn.transform_iats_schedule(iats, d, T)[0],
      "iats": iats,
      "iats_diff": np.diff(iats),
      "loss": loss,
      "calc_time": toc - tic
    }
  results.append(result)
  print(f'schedule: {result["schedule"]},\niats: {iats},\niats_diff: {result["iats_diff"]},\nloss = {result["loss"]}, \
  calc time: {result["calc_time"]} secs')


In [ ]:
# Get start time and set timer seconds
start_time = time.time()
seconds = 20
bestresult = ""

# Start timer while loop
while True:
    current_time = time.time()
    elapsed_time = current_time - start_time
    
    if elapsed_time > seconds:
        print("Finished iterating in: " + str(int(elapsed_time))  + " seconds")
        break
      
    ## Start iterator for inter arrival times in shift matrix
    results = []
    for iats in iats_sm:
      tic = time.time()
      simreswt = [None] * nsims # array for saving waiting times
      simresln = 0 # array for saving lateness
      
      for i in range(nsims):
        experiment = fn.simulate_crn(iats, ctsm[i], stsm[i], logs=logs)
        simreswt[i] = experiment[0].mean()
        ln = max(0, (experiment[1] - d))
        simresln += ln
        
      mwt = np.array(simreswt).mean()
      mln = simresln / nsims
      loss = 0.5 * mwt + 0.5 * mln
      toc = time.time()
      result = {
          "schedule": fn.transform_iats_schedule(iats, d, T)[0],
          "iats": iats,
          "iats_diff": np.diff(iats),
          "loss": loss,
          "calc_time": toc - tic
        }
      if (bestresult == "" || result["loss"] < bestresult):
            bestresult = result["loss"]
            print(bestresult)
      
      results.append(result)
      print(f'schedule: {result["schedule"]},\niats: {iats},\niats_diff: {result["iats_diff"]},\nloss = {result["loss"]}, \
  calc time: {result["calc_time"]} secs')
      
### Simulate item of the shift matrix

#### If loss is lower than previous save iats vector and loss

## Calculate new shift matrix based upon best iats vector